In [1]:
import pandas as pd

In [2]:
df_PCPN = pd.read_csv('PCPN.csv')
df_TMAX = pd.read_csv('TMAX.csv')
df_TAVG = pd.read_csv('TAVG.csv')
df_TMAX = df_TMAX.rename(columns={'Unnamed: 0': 'ID', 'Unnamed: 1': 'Janvier', 'Unnamed: 2': 'Février', 'Unnamed: 3': 'Mars', 'Unnamed: 4': 'Avril', 'Unnamed: 5': 'Mai', 'Unnamed: 6': 'Juin', 'Unnamed: 7': 'Juillet', 'Unnamed: 8': 'Août', 'Unnamed: 9': 'Septembre', 'Unnamed: 10': 'Octobre', 'Unnamed: 11': 'Novembre', 'Unnamed: 12': 'Décembre'})

In [3]:
#Example of a retrieved DataFrame :
df_PCPN.head()

,ID,Janvier,Février,Mars,Avril,Mai,Juin,Juillet,Août,Septembre,Octobre,Novembre,Décembre
0,1001011895,7.03,2.96,8.36,3.53,3.96,5.40,3.92,3.36,0.73,2.03,1.44,3.66
1,1001011896,5.86,5.42,5.54,3.98,3.77,6.24,4.38,2.57,0.82,1.66,2.89,1.94
2,1001011897,3.27,6.63,10.94,4.35,0.81,1.57,3.96,5.02,0.87,0.75,1.84,4.38
3,1001011898,2.33,2.07,2.60,4.56,0.54,3.13,5.80,6.02,1.51,3.21,6.66,3.91
4,1001011899,5.80,6.94,3.35,2.22,2.93,2.31,6.80,2.90,0.63,3.02,1.98,5.25


In [6]:
#Creating dataframes:

def cleaning_df(df,string):
    
    #First, we split the 10-digit identifier of each line.
    #We obtain the year, month, state and county of each measurement.
    
    df['ID'] = df.ID.astype(str).str.zfill(11)
    
    df['Etat'] = df.ID.astype(str).apply(lambda x:x[:2])
    
    df['Comté'] = df.ID.astype(str).apply(lambda x:x[2:5])

    df['Année'] = df.ID.astype(str).apply(lambda x:x[-4:])
    
    #We no longer need the ID column. 
    #To merge the three dataframes, we'll use the other columns.
    df = df.drop('ID', axis = 1)
       
    
    Liste_mois = ['Janvier','Février','Mars','Avril','Mai','Juin','Juillet','Août','Septembre','Octobre','Novembre','Décembre']
    df = pd.melt(df,['Comté','Année','Etat'],Liste_mois)

    df = df.rename(columns={'value':string,'variable' : 'Mois'})
    
    df = df.loc[(df['Année'].astype(int) > 1991) & (df['Année'].astype(int) < 2016)]
    
    return df


df_PCPN_clean = cleaning_df(df_PCPN,'PCPN')
df_TMAX_clean = cleaning_df(df_TMAX,'TMAX')
df_TAVG_clean = cleaning_df(df_TAVG,'TAVG')




In [7]:
#Example of transformed dataframe: 
df_TMAX_clean.head()

,Comté,Année,Etat,Mois,TMAX
97,001,1992,01,Janvier,54.5
98,001,1993,01,Janvier,59.3
99,001,1994,01,Janvier,52.2
100,001,1995,01,Janvier,57.0
101,001,1996,01,Janvier,56.0


In [16]:
#Merging of the three dataframes :

df_deux = pd.merge(left =df_TMAX_clean, right=df_TAVG_clean, how="inner", left_on=['Comté','Année','Etat','Mois'], right_on=['Comté','Année','Etat','Mois'])
df_ALL = pd.merge(left =df_deux, right=df_PCPN_clean, how="inner", left_on=['Comté','Année','Etat','Mois'], right_on=['Comté','Année','Etat','Mois'])
df_ALL.Année = df_ALL.Année.astype('int64')

In [6]:
#Creating a dataframe for export:
df_ALL.to_csv('df_TMAX_TAVG_PCPN')


In [9]:
df_init = pd.read_csv('df_initial.csv',usecols=['FIRE_YEAR','DISCOVERY_DATE','FIPS_CODE','STATE','FIRE_SIZE','COUNTY'])

def julian_to_gregorian(julian_date):
    return pd.to_datetime(julian_date, origin='julian', unit='D').strftime('%d/%m/%Y')

# Apply the conversion to the DISCOVERY_DATE column
df_init['DISCOVERY_DATE'] = df_init['DISCOVERY_DATE'].apply(julian_to_gregorian)

C:\Users\pierr\AppData\Local\Temp\ipykernel_18052\4040289050.py:1: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_init = pd.read_csv('df_initial.csv',usecols=['FIRE_YEAR','DISCOVERY_DATE','FIPS_CODE','STATE','FIRE_SIZE','COUNTY'])


In [10]:
#This manoeuvre causes numerous warnings which are not important for the rest of the work.import warnings
warnings.filterwarnings('ignore')
df_init.DISCOVERY_DATE = pd.to_datetime(df_init.DISCOVERY_DATE).dt.month

In [11]:
df_init = df_init.rename(columns={"DISCOVERY_DATE":"Mois","FIRE_YEAR":"Année",'STATE':'Etat','COUNTY':'Comté'})


Each county must then be given its own code, consisting of its state and its FIPS code.
This is necessary because counties can share the same name from one state to another. 

In [18]:
#Creating lists for replacement. 
Liste_ID = ['01','02','03','04','05','06','07','08','09', 
            '10', '11','12','13','14','15','16','17','18','19', '20',
            '21','22','23','24','25','26','27','28','29', '30',
            '31','32','33','34','35','36','37','38','39', '40',
            '41','42','43','44','45','46','47','48','50']

Liste_Etats = ['AL','AZ','AR','CA','CO','CT','DE','FL','GA','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO',
              'MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY','AK']


df_init.FIPS_CODE = df_init.FIPS_CODE.fillna(999)
df_init.Etat = df_init.Etat.replace(Liste_Etats,Liste_ID)

df_init.Comté = df_init.Etat.astype(str) + '-' + round(df_init.FIPS_CODE,0).astype(int).astype(str).str.zfill(3)

df_ALL.Mois = df_ALL.Mois.replace(['Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin', 'Juillet',
       'Août', 'Septembre', 'Octobre', 'Novembre', 'Décembre'],[1,2,3,4,5,6,7,8,9,10,11,12])
df_ALL['Etat_Comté'] = df_ALL.Etat.astype(str) + '-' + df_ALL.Comté.astype(str)
df_ALL.head()


,Comté,Année,Etat,Mois,TMAX,TAVG,PCPN,Etat_Comté
0,001,1992,01,1,54.5,44.6,5.62,01-001
1,001,1993,01,1,59.3,49.7,6.08,01-001
2,001,1994,01,1,52.2,41.7,3.54,01-001
3,001,1995,01,1,57.0,46.4,4.95,01-001
4,001,1996,01,1,56.0,44.0,7.04,01-001


We can now merge the created dataframe with the original dataframe.

In [13]:
df_init.head()

,Année,Mois,FIRE_SIZE,Etat,Comté,FIPS_CODE
0,2005,2,0.10,04,04-063,63.0
1,2004,12,0.25,04,04-061,61.0
2,2004,5,0.10,04,04-017,17.0
3,2004,6,0.10,04,04-003,3.0
4,2004,6,0.10,04,04-003,3.0


In [14]:
df_init_groupé = df_init.groupby(['Année','Mois','Comté'],as_index=False).agg({'FIRE_SIZE':'sum','FIPS_CODE':'count'})
df_init_groupé.head()

,Année,Mois,Comté,FIRE_SIZE,FIPS_CODE
0,1992,1,01-999,65.1,13
1,1992,1,02-003,17.7,4
2,1992,1,02-009,1.0,1
3,1992,1,02-013,163.1,4
4,1992,1,02-015,57.5,2


In [20]:
df_FINAL = pd.merge(left = df_ALL, right = df_init_groupé, how = 'left', left_on = ['Année','Mois','Etat_Comté'], right_on = ['Année','Mois','Comté'])

df_FINAL = df_FINAL.drop(['Comté_x','Comté_y'],axis=1)
df_FINAL.FIPS_CODE = df_FINAL.FIPS_CODE.fillna(0).astype(int)
df_FINAL.FIRE_SIZE = df_FINAL.FIRE_SIZE.fillna(0)
df_FINAL['Comté'] = df_FINAL.Etat_Comté.apply(lambda x:x[3:])
df_FINAL = df_FINAL.rename(columns={'FIPS_CODE':'Nombre de feux','FIRE_SIZE':'Etendue totale des feux'})
df_FINAL.head()

,Année,Etat,Mois,TMAX,TAVG,PCPN,Etat_Comté,Etendue totale des feux,Nombre de feux,Comté
0,1992,01,1,54.5,44.6,5.62,01-001,0.0,0,001
1,1993,01,1,59.3,49.7,6.08,01-001,0.0,0,001
2,1994,01,1,52.2,41.7,3.54,01-001,19.6,3,001
3,1995,01,1,57.0,46.4,4.95,01-001,2.6,4,001
4,1996,01,1,56.0,44.0,7.04,01-001,0.0,0,001


In [109]:
df_FINAL.to_csv('Dataframe_Final_Comtés_Mois.csv')